# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [37]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [38]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-cartier,50.0334,-66.8654,267.21,85,100,3.14,CA,2023-02-08 01:45:35
1,1,tuktoyaktuk,69.4541,-133.0374,241.15,75,100,1.54,CA,2023-02-08 01:41:01
2,2,atuona,-9.8000,-139.0333,299.42,74,28,7.37,PF,2023-02-08 01:45:44
3,3,avarua,-21.2078,-159.7750,302.18,74,28,5.14,CK,2023-02-08 01:45:44
4,4,klaksvik,62.2266,-6.5890,281.66,84,100,25.39,FO,2023-02-08 01:43:55


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [39]:
%%capture --no-display

City_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity"
)

# Display the map plot
City_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [64]:
# Narrow down cities that fit criteria and drop any results with null values
weather_city = city_data_df.loc[(city_data_df["Max Temp"] <= 305)& (city_data_df["Max Temp"] >= 290), ["Max Temp"]]
# & (city_data_df['Humidity'] <= 40)]

# Drop any rows with null values
weather_city = city_data_df.dropna(how='any',axis=0)

# Display sample data
weather_city.count()

City_ID       536
City          536
Lat           536
Lng           536
Max Temp      536
Humidity      536
Cloudiness    536
Wind Speed    536
Country       536
Date          536
dtype: int64

### Step 3: Create a new DataFrame called `hotel_df`.

In [35]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(weather_city)
hotel_df.pop("City_ID")
hotel_df.pop('Max Temp')
hotel_df.pop('Cloudiness')
hotel_df.pop('Wind Speed')
hotel_df.pop('Date')
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Lat,Lng,Humidity,Country,Hotel Name
0,port-cartier,50.0334,-66.8654,85,CA,
1,tuktoyaktuk,69.4541,-133.0374,75,CA,
2,atuona,-9.8000,-139.0333,74,PF,
3,avarua,-21.2078,-159.7750,74,CK,
4,klaksvik,62.2266,-6.5890,84,FO,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"airport",
    "apiKey": geoapify_key,
    "limit":1
}

# Set the base URL
base_url = "https://api.geoapify.com/v1/geocode/search"

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row[1]
    lon = row[2]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    "Hotel Name",
    "Country"
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity"
)

# Display the map plot
hotel_plot